In [1]:
print('Set up started')
from qiskit_textbook.games import hello_quantum
print('Set up completed')

Set up started
Set up completed


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# For Not gate
initialize = []
success_condition = {}
allowed_gates = {'0':{'NOT':3}, '1': {}, 'both': {}}
vi = [[1], False, False]
qubit_names = {'0':'the only bit', '1': None}
puzzle = hello_quantum.run_game(initialize, success_condition, allowed_gates, vi, qubit_names)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01\xb0\x08\x06\x00\x00\x00\xc8N0*\x…

In [5]:
initialize = []
success_condition = {}
allowed_gates = {'0':{}, '1':{'NOT':0}, 'both':{}}
vi = [[], False, False]
qubit_names = {'0':'the bit on the left', '1': 'the bit on the right'}
puzzle = hello_quantum.run_game(initialize, success_condition, allowed_gates, vi, qubit_names)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01\xb0\x08\x06\x00\x00\x00\xc8N0*\x…

In [6]:
# For CNOT Gate
initialize = [['x','0']]
success_condition = {'IZ':-1.0}
allowed_gates = {'0':{'CNOT':0},'1':{'CNOT':0},'both':{}}
vi = [[], False, False]
qubit_names = {'0':'the bit on the left', '1': 'the bit on the right'}
puzzle = hello_quantum.run_game(initialize, success_condition, allowed_gates, vi, qubit_names)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01\xb0\x08\x06\x00\x00\x00\xc8N0*\x…

In [7]:
initialize = [['x','0']]
success_condition = {'ZI':1.0, 'IZ':-1.0}
allowed_gates = {'0':{'CNOT':0}, '1': {'CNOT':0}, 'both':{}}
vi = [[],False, False]
qubit_names = {'0':'the bit on the left','1':'the bit on the right'}
puzzle = hello_quantum.run_game(initialize, success_condition, allowed_gates, vi, qubit_names)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01\xb0\x08\x06\x00\x00\x00\xc8N0*\x…

In [2]:
initialize = [['h', '0']]
success_condition = {'IZ': 0.0}
allowed_gates = {'0': {'CNOT': 0}, '1': {'CNOT': 0}, 'both': {}}
vi = [[], False, False]
qubit_names = {'0':'the bit on the left', '1':'the bit on the right'}
puzzle = hello_quantum.run_game(initialize, success_condition, allowed_gates, vi, qubit_names)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01\xb0\x08\x06\x00\x00\x00\xc8N0*\x…

In [3]:
# Bell test for Quantum Variable
'''Bell's inequality with Quantum variable in this section we are going to take Quantum variable for our 
experiment and also going to generate a video game'''
from qiskit_textbook.games import hello_quantum # To generate puzzle game
from qiskit import assemble, transpile #to generate task to the quantum computer
from qiskit import *
import numpy as np
from qiskit import Aer

'''Initializing qubit for our operation'''

def initialize_program():
    qubit = QuantumRegister(2)
    A = qubit[0]
    B = qubit[1]
    
    bit = ClassicalRegister(2)
    a = bit[0]
    b = bit[1]
    
    qc = QuantumCircuit(qubit, bit) 
    
    return A, B, a, b, qc
    
#initialize_program()    

'''Initializing hash function so that we could get our value into bit '''
    
def hash2bit(variable, hash_type, bit, qc):
    if hash_type == 'H':
        qc.h(variable)
    qc.measure(variable, bit)
    
'''Here we are initializing the game and also running the run.puzzle game'''    
    
initialize = []
success_condition = {'ZZ': +0.7071, 'ZX':+0.7071, 'XZ':+0.7071, 'XX':-0.7071}
allowed_gates = {'0':{'bloch':0, 'x':0, 'z':0, 'h':0, 'cx':0, 'ry(pi/4)':0}, '1':{'bloch':0, 'x':0, 'z':0, 'h':0, 'cx':0, 'ry(-pi/4)':0}, 'both':{'cz':0, 'unbloch':0}}
vi = [[], True, False]
qubit_games = {'0':'A', '1': 'B'}
puzzle = hello_quantum.run_game(initialize, success_condition, allowed_gates, vi, qubit_games, mode = 'line')

'''Here we are calling  values from that puzzle and using it to setup our variable'''

def setup_variables(A, B, qc):
    for line in puzzle.program:
        eval(line)
        

shots = 8192

def calculate_P ( backend ):
    
    P = {}
    program = {}
    for hashes in ['VV','VH','HV','HH']:

        A, B, a, b, program[hashes] = initialize_program ()

        setup_variables( A, B, program[hashes] )

        hash2bit ( A, hashes[0], a, program[hashes])
        hash2bit ( B, hashes[1], b, program[hashes])
            
    # submit jobs
    t_qcs = transpile(list(program.values()), backend)
    qobj = assemble(t_qcs, shots=shots)
    job = backend.run(qobj)

    # get the results
    for hashes in ['VV','VH','HV','HH']:
        stats = job.result().get_counts(program[hashes])
        
        P[hashes] = 0
        for string in stats.keys():
            a = string[-1]
            b = string[-2]
            
            if a!=b:
                P[hashes] += stats[string] / shots

    return P

device = 'aer_simulator'
backend = Aer.get_backend(device)

P = calculate_P(backend)
print(P)

def bell_test (P):
    
    sum_P = sum(P.values())
    for hashes in P:
        
        bound = sum_P - P[hashes]
        
        print("The upper bound for P['"+hashes+"'] is "+str(bound))
        print("The value of P['"+hashes+"'] is "+str(P[hashes]))
        if P[hashes]<=bound:
            print("The upper bound is obeyed :)\n")
        else:
            if P[hashes]-bound < 0.1:
                print("This seems to have gone over the upper bound, but only by a little bit :S\nProbably just rounding errors or statistical noise.\n")
            else:
                print("!!!!! This has gone well over the upper bound :O !!!!!\n")

bell_test(P)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb0\x00\x00\x01\xb0\x08\x06\x00\x00\x00\xc8N0*\x…

{'VV': 0, 'VH': 0.50244140625, 'HV': 0.5015869140625, 'HH': 0.5001220703125}
The upper bound for P['VV'] is 1.504150390625
The value of P['VV'] is 0
The upper bound is obeyed :)

The upper bound for P['VH'] is 1.001708984375
The value of P['VH'] is 0.50244140625
The upper bound is obeyed :)

The upper bound for P['HV'] is 1.0025634765625
The value of P['HV'] is 0.5015869140625
The upper bound is obeyed :)

The upper bound for P['HH'] is 1.0040283203125
The value of P['HH'] is 0.5001220703125
The upper bound is obeyed :)

